# FAST-UAV - Life Cycle Assessments of multirotor UAVs

In this notebook, the LCA plugin of FAST-OAD is used to evaluate the environmental impacts of a multirotor drone and optimize its sizing based on environmental criteria.

## 1. Setting up the problem

In [ ]:
# Import required librairies
import os.path as pth
import sys
import logging
import fastoad.api as oad
import shutil
import matplotlib.pyplot as plt
from fastuav.utils.postprocessing.analysis_and_plots import multirotor_geometry_plot, mass_breakdown_sun_plot_drone

sys.path.append(pth.abspath("."))

#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.display import display, HTML, IFrame
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Declare paths to folders and files
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE_LCA = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_lca.yaml")
CONFIGURATION_FILE_MDO_LCA = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_mdo_lca.yaml")
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_inputs_lca.xml")

# For having log messages display on screen
# logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

## 2. Run the LCA on an existing design

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE_LCA, SOURCE_FILE, overwrite=True)
oad.variable_viewer(input_file)

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE_LCA, overwrite=True)

In [ ]:
output_file = eval_problem.output_file_path
shutil.copy(output_file, 
            pth.join(SOURCE_FOLDER_PATH, 'problem_outputs_lca.xml')
)

In [ ]:
oad.variable_viewer(output_file)

## 3. Optimize the sizing based on environmental criteria

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE_MDO_LCA, SOURCE_FILE, overwrite=True)
oad.variable_viewer(input_file)

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE_MDO_LCA, overwrite=True)

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE_MDO_LCA)

In [ ]:
output_file = optim_problem.output_file_path
shutil.copy(output_file, 
            pth.join(SOURCE_FOLDER_PATH, 'problem_outputs_mdo_lca.xml')
)

In [ ]:
oad.variable_viewer(output_file)

In [ ]:
res

In [ ]:
from fastoad.io import VariableIO
from fastuav.constants import LCA_PARAM_KEY, LCA_DEFAULT_METHOD
import time

# Select model
model = get_lca_main_activity()  # top-level model

# Get parameters values from problem outputs
variables = VariableIO(OUTPUT_FILE).read()
param_names = [p for p in variables.names() if p.startswith(LCA_PARAM_KEY)]
parameters = {}
for p in param_names:
    parameters[p.replace(LCA_PARAM_KEY, "")] = variables[p].value[0]

# Run Monte Carlo
methods = [eval(m) for m in LCA_DEFAULT_METHOD]
res = lca_monte_carlo(
    model, # the model
    methods, # impacts to assess 

    # Number of Monte Carlo runs
    n_runs=1000, 
    
    # Whether uncertainty on characterization factors is taken into account or not
    cfs_uncertainty = False,

    # Parameters of the model
    **parameters
)
res.to_csv("./workdir/lca_monte_carlo.csv")

In [ ]:
from fastoad.io import VariableIO
from fastuav.constants import LCA_PARAM_KEY, LCA_DEFAULT_METHOD
import time

# Select model
model = get_lca_main_activity()  # top-level model

# Get parameters values from problem outputs
variables = VariableIO(OUTPUT_FILE).read()
param_names = [p for p in variables.names() if p.startswith(LCA_PARAM_KEY)]
parameters = {}
for p in param_names:
    parameters[p.replace(LCA_PARAM_KEY, "")] = variables[p].value[0]

# Run Monte Carlo
methods = [eval(m) for m in LCA_DEFAULT_METHOD]

In [ ]:
res.describe()

In [ ]:
df = pd.read_csv("./workdir/lca_monte_carlo.csv", header=[0,1,2], index_col=0)

In [ ]:
df.quantile([0.025, 0.975])

In [ ]:
df.median()

In [ ]:
df.describe()

In [ ]:
# Plot distributions
fig = make_subplots(rows=4, cols=4, subplot_titles=[s[1].replace(':', '<br>') for s in df.columns.tolist()], horizontal_spacing = 0.08, vertical_spacing = 0.13)
axes_units=[r"$mol H^+_{eq}$", r"$kgCO_{2eq}$", '$CTU_e$', "$MJ$", r"$kgP_{eq}$", r"$kgN_{eq}$", r"$mol N_{eq}$", "$CTUh$", "$CTUh$", r"$kBq U235_{eq}$", r"$\text{soil quality index}$", r"$kg Sb_{eq}$", r"$kg CFC-11_{eq}$", r"$\text{disease incidence}$", r"$kg NMVOC_{eq}$", r"$m^3 \text{ world eq. deprived}$"]
deterministic_values = [4.741456481, 502.6635915, 3756.514583, 9683.489122, 0.434746217, 0.608561593, 5.591831523, 1.53206E-06, 2.67813E-05, 214.0044574, 2328.077792, 0.029507826, 1.39928E-05, 2.56764E-05, 1.759051648, 630.7958567]

for i in range(len(df.columns)): 
    fig.add_trace(
        go.Histogram(x=df.iloc[:, i], histnorm='probability'),
        row=i // 4 + 1, 
        col=i % 4 + 1,
    )
    fig.add_vline(x=deterministic_values[i], line_dash = 'dash',
                  row=i // 4 + 1, 
                  col=i % 4 + 1,
                 )
    fig.update_xaxes(title_text=axes_units[i], row=i // 4 + 1, col=i % 4 + 1, titlefont=dict(size=14))
    
for i in range(4):
    for j in range(3):
        fig.update_yaxes(showticklabels=True, matches=f'y{4*i+j+2}', row=i+1, col=j+1)
    
fig.update_layout(title=None, width=1200, height=1200, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_traceorder="reversed", showlegend=False)
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', titlefont=dict(size=14))
fig.update_yaxes(title='probability', col=1)
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
from fastuav.utils.postprocessing.sensitivity_analysis.sensitivity_analysis import doe_fast

# Inputs
x_dict = {
    "lca:parameters:n_cycles_uav": np.linspace(1.0, 10000.0, 1000),
         }

# Outputs
y_list = [
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:airframe:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:batteries:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:controllers:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:motors:production',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:payload',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:payload:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers:efficiency',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers:mass',
    'lca:postprocessing:aggregation:weighted_single_score:model_per_FU:propellers:production',
]

# Results of DoE
df = doe_fast("list", x_dict, y_list, CONFIGURATION_FILE_LCA_ONLY)  # Run DoE on LCA model
df.to_csv('./workdir/df_components_contributions.csv')

In [ ]:
import matplotlib

df.columns = df.columns.str.replace('lca:postprocessing:aggregation:weighted_single_score:', '')
component_name = 'batteries'

color = px.colors.qualitative.Plotly[2]
opacities = [0.2, 0.5, 1.0]
patterns = ['x', '.', '/']
fig = px.area(df, x='lca:parameters:n_cycles_uav', 
              y=['model_per_FU:' + component_name + ':mass', 
                 'model_per_FU:' + component_name + ':efficiency',
                 'model_per_FU:' + component_name + ':production'
                ], 
              groupnorm='fraction',
              color_discrete_sequence=[color])
newnames = {'model_per_FU:' + component_name + ':efficiency':'efficiency', 'model_per_FU:' + component_name + ':mass':'mass', 'model_per_FU:' + component_name + ':production':'production'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
for idx in range(len(fig.data)):
    fig.data[idx].fillcolor = 'rgba' + str(matplotlib.colors.to_rgb(color) + (opacities[idx],)).replace('1.0', '0.99')
    fig.data[idx].line.width = 1.2
    fig.data[idx].line.color = 'black'
    fig.data[idx].fillpattern.shape = patterns[idx]
fig.add_annotation(x=9500, y=0.6,
            text="production",
            showarrow=True,
            ax=80,
            ay=0,
            arrowsize=1.5,
            arrowhead=1)
fig.add_annotation(x=9500, y=0.06,
            text="efficiency",
            showarrow=True,
            ax=80,
            ay=-40,
            arrowsize=1.5,
            arrowhead=1)
fig.add_annotation(x=9500, y=0.015,
            text="mass",
            showarrow=True,
            ax=65,
            ay=-10,
            arrowsize=1.5,
            arrowhead=1)
fig.update_layout(title=None, width=600, height=350, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title="", legend_traceorder="reversed", showlegend=False)
fig.update_xaxes(title='Number of cycles (-)', showline=True, linewidth=1, linecolor='black', tickformat=".f", showgrid=False, gridwidth=0.0, tickvals=[0, 2000, 4000, 6000, 8000, 10000], ticks="outside") #, range=[0,10000])
fig.update_yaxes(title='Single score', showline=True, linewidth=1, linecolor='black', tickformat='.0%', showgrid=False, gridwidth=0.0, range=[0,1.0])
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf', scale=5)

In [ ]:
# COMPARISON OF THE LCA SCORES

OUTPUT_FILE_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_nmc811.xml")
OUTPUT_FILE_LFP = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_lfp.xml")
OUTPUT_FILE_SI_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_si_nmc.xml")

output_files_dict = {'G/NMC': OUTPUT_FILE_NMC,
                     'G/LFP': OUTPUT_FILE_LFP,
                     'Si/NMC': OUTPUT_FILE_SI_NMC
                    }
colors = [px.colors.qualitative.Set1[8], px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]]

fig = go.Figure()

idx = 0
for name, output_file in output_files_dict.items():
    fig_data = lca_plot(output_file, result_step = 'weighting', filter_option = 'default', filter_level = 0, percent=False).data[0]
    #fig.add_trace(go.Bar(x=fig_data.labels, y=fig_data.values))
    fig.add_trace(go.Bar(x=fig_data.x, y=fig_data.y, name=name, marker_color=colors[idx]))
    idx += 1

fig.update_layout(barmode='group')
fig.update_layout(title=None, width=850, height=650, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title_text='Battery chemistry',
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Points', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>", 2)[1] for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

#for idx in range(len(fig.data)):
#    fig.data[idx].x = [s.split("<br>")[1] + " " + s.split("<br>")[3] for s in fig.data[idx].x]
#    fig.data[idx].marker.line.width = 0
    
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# COMPARISON OF THE LCA SCORES - NORMALIZED W.R.T BASELINE

OUTPUT_FILE_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_nmc811.xml")
OUTPUT_FILE_LFP = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_lfp.xml")
OUTPUT_FILE_SI_NMC = pth.join(WORK_FOLDER_PATH, "problem_outputs_lca_si_nmc.xml")

output_files_dict = {'G/NMC': OUTPUT_FILE_NMC,
                     'G/LFP': OUTPUT_FILE_LFP,
                     'Si/NMC': OUTPUT_FILE_SI_NMC
                    }
colors = [px.colors.qualitative.Set1[8], px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]]

fig = go.Figure()

fig_data_ref = lca_plot(OUTPUT_FILE_NMC, result_step = 'characterization', filter_option = 'default', filter_level = 0, percent=False).data[0]

idx = 0
for name, output_file in output_files_dict.items():
    fig_data = lca_plot(output_file, result_step = 'characterization', filter_option = 'default', filter_level = 0, percent=False).data[0]
    #fig.add_trace(go.Bar(x=fig_data.labels, y=fig_data.values))
    fig.add_trace(go.Bar(x=fig_data.x, y=np.asarray(fig_data.y)/np.asarray(fig_data_ref.y), name=name, marker_color=colors[idx]))
    idx += 1

fig.update_layout(barmode='group')
fig.update_layout(title=None, width=850, height=650, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), legend_title_text='Battery chemistry',
                 margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=90)
fig.update_yaxes(title='Relative score', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>", 2)[1] for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

#for idx in range(len(fig.data)):
#    fig.data[idx].x = [s.split("<br>")[1] + " " + s.split("<br>")[3] for s in fig.data[idx].x]
#    fig.data[idx].marker.line.width = 0
    
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
# VARIATION WITH NUMBER OF CYCLES
df_nmc = pd.read_csv("./workdir/df_cycles_single_scores_nmc.csv")
df_lfp = pd.read_csv("./workdir/df_cycles_single_scores_lfp.csv")
df_si_nmc = pd.read_csv("./workdir/df_cycles_single_scores_si_nmc.csv")

df_nmc['total'] = df_nmc['operation'] + df_nmc['batteries'] + df_nmc['propellers'] + df_nmc['controllers'] + df_nmc['airframe'] + df_nmc['motors']
df_lfp['total'] = df_lfp['operation'] + df_lfp['batteries'] + df_lfp['propellers'] + df_lfp['controllers'] + df_lfp['airframe'] + df_lfp['motors']
df_si_nmc['total'] = df_si_nmc['operation'] + df_si_nmc['batteries'] + df_si_nmc['propellers'] + df_si_nmc['controllers'] + df_si_nmc['airframe'] + df_si_nmc['motors']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_nmc['n_cycles'], y=df_nmc['total'], line=dict(color=px.colors.qualitative.Dark2[7], dash='solid'), name='G/NMC'))
fig.add_trace(go.Scatter(x=df_lfp['n_cycles'], y=df_lfp['total'], line=dict(color=px.colors.qualitative.Plotly[1], dash='solid'), name='G/LFP'))
fig.add_trace(go.Scatter(x=df_si_nmc['n_cycles'], y=df_si_nmc['total'], line=dict(color=px.colors.qualitative.Plotly[0], dash='solid'), name='Si/NMC'))

fig.update_layout(title=None, width=600, height=300, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=14), 
                  legend_title_text='Battery chemistry', legend_traceorder="reversed", margin=dict(l=10, r=10, t=0, b=0))
fig.update_xaxes(title='Number of cycles (-)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', showgrid=True, gridwidth=0.01, tickformat=".f", range=[0,2600])
fig.update_yaxes(title='Single score (points)', showline=True, linewidth=1, linecolor='black', gridcolor='lightgrey', showgrid=True, gridwidth=0.01, range=[0,0.17])
#fig.for_each_trace(lambda trace: trace.update(fillcolor = trace.line.color, marker_line_color = 'black', marker_line_width = 1.0))
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')